# Vorbereitung

In [1]:
%env KERAS_BACKEND=theano
%env THEANO_FLAGS=floatX=float32,device=cpu

env: KERAS_BACKEND=theano
env: THEANO_FLAGS=floatX=float32,device=cpu


In [2]:
%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

import pandas as pd
import time
import datetime

import os
import glob

Using Theano backend.


In [3]:
from tqdm import tqdm_notebook # progress bar

import math
import numpy as np
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

# Run

In [4]:
from bb_behavior.tracking.pipeline import get_default_pipeline
default_pipeline = None
default_pipeline = get_default_pipeline(localizer_threshold="0.50")

/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [5]:
# Hilfsfunktionen
def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[1]

def datestring_to_filename(datestring, prefix="e00_"):
    return config["videos_dir"] + prefix + datestring + ".h264"

def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))

In [6]:
# get all videos between timestamp_in and timestamp_out
def get_videos_between(timestamp_in, timestamp_out, all_paths):
    """ returns all video between timestamp_in and timestamp out (inclusive) """
    
    mask = (all_paths['video'] >= timestamp_in) & (all_paths['video'] <= timestamp_out)
    return list(all_paths[mask]['video'])

def loadGTD(path):
    # read in the test data csv
    test_data = pd.read_csv(path)

    # convert the full filenames to string timestamps, and sort by timestamp_in
    test_data['video'] = test_data['video'].apply(lambda x: filename_to_datestring(x))
    test_data['video_start_time'] = test_data['timestamp_in'].apply(lambda x: filename_to_datestring(x))
    test_data['video_end_time'] = test_data['timestamp_out'].apply(lambda x: filename_to_datestring(x))
    test_data.sort_values(['video_start_time'], inplace=True)

    test_data.drop('timestamp_in', 1, inplace=True)
    test_data.drop('timestamp_out', 1, inplace=True)
    test_data.drop('video', 1, inplace=True)
    return test_data

In [10]:
tmp = loadGTD("bees_test.csv")
tmp

,bee_id,in_direction,out_direction,video_start_time,video_end_time
102,1970,left,left,2018-08-19-02-21-37,2018-08-19-02-22-40
0,855,right,left,2018-08-19-05-25-51,2018-08-19-05-25-51
1,220,left,right,2018-08-19-05-25-51,2018-08-19-05-25-51
24,8692,left,right,2018-08-19-06-35-41,2018-08-19-06-35-41
106,1191,right,left,2018-08-19-07-10-38,2018-08-19-07-10-38
107,1671,right,left,2018-08-19-07-10-38,2018-08-19-07-10-38
56,1788,right,left,2018-08-19-07-16-49,2018-08-19-07-16-49
55,749,left,right,2018-08-19-07-16-49,2018-08-19-07-16-49
43,3925,left,right,2018-08-19-07-47-52,2018-08-19-07-47-52
89,481,left,right,2018-08-19-08-21-08,2018-08-19-08-21-29


In [11]:
config = dict(tag_pixel_diameter=50,
              n_frames=None,
              confidence_filter_detections=0.08,
              confidence_filter_tracks=0.20,
              coordinate_scale=1.0,
              start_time=None,
              fps=10.0,
              cam_id=0,
              left_leaving_area = 0.3, # Prozente vom Bildschirmrand, zB. bei 1000px und 0.15 -> 0-150px
              right_leaving_area = 0.3,
              px_x_resolution_vid = 1944,
              videos_dir = "../videos/")


In [16]:
def detect_tracks(paths, save_to_csv = False):
    # Hier passiert das eigentliche Tracken und speichern der Ergebnisse:
    num_processed_videos = 0
    video_data = dict()
    frame_info = None
    detections = None
    bad_paths = []

    for path in tqdm_notebook(paths):
        start_time = config["start_time"]
        cam_id = config["cam_id"]
        try:
            num_processed_videos += 1

            frame_info, detections = detect_markers_in_video(path,
                                                          decoder_pipeline=default_pipeline,#pipeline=pipelines(),
                                                         tag_pixel_diameter=config["tag_pixel_diameter"],
                                                          n_frames=config["n_frames"],
                                                          fps=config["fps"],
                                                         progress="tqdm_notebook"
                                                )
            # Sonst würden keine Tracks erkannt werden -> Fehlermeldung
            if len(detections[detections['confidence']>=config["confidence_filter_detections"]]) == 0:
                continue
            tracks = track_detections_dataframe(detections,
                                                tracker="tracker.det_score_fun.frag_score_fun.dill",
                                                confidence_filter_detections=config["confidence_filter_detections"],
                                               confidence_filter_tracks=config["confidence_filter_tracks"],
                                                coordinate_scale=config["coordinate_scale"],
                                               )
            date_string = filename_to_datestring(path)
            tracks['video'] = date_string
            video_data[path] = (frame_info, detections, tracks)
        except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
            try:
                bad_paths.append(path)
                # wir arbeiten später nochmal mit paths, daher müssen das leere löschen, weil sonst
                # in video_data kein zugehöriger Value zu Key = file zu finden ist.
            except KeyError as err:
                continue
        except Exception as err:
            print(err)
            raise
        # only first vid: break

    for bad_path in bad_paths:
        paths.remove(bad_path)

    #video_data["file"][0] --> frame-info, [1] --> detections, [2] --> tracks
    for path in paths:
        display_tracking_results(path, video_data[path][0], video_data[path][1], video_data[path][2])

    tracks = [video_data[paths[x]][2] for x in range(len(paths))]
    tracks = pd.concat(tracks,ignore_index=True)
    tracks = tracks.drop(columns=["localizerSaliency", "beeID", "camID", "frameIdx"])

    if save_to_csv:
        with open("tracks.csv", "w") as f:
            tracks.to_csv(f)

    return tracks

## a: Create video path list from all videos in videos_dir

In [12]:
# Einen Iterable speichern, der alle Videos in einem Iterable zur Verfügung stellt
# Diesen Iterable können wir dann in der nächsten Zelle mit tqdm schön durchlaufen
base_directory = config["videos_dir"]
paths = [i for i in os.listdir(base_directory) if i.endswith(".h264")]
for i in range(len(paths)):
    paths[i] = base_directory + paths[i]

## b: Create video path list from test set

In [14]:
# read in the test data csv
test_data = loadGTD("bees_test.csv")

all_videos = []

all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
all_paths.sort_values(['video'])

# go through test_data and get all videos between timestamp_in and timestamp_out
for index, row in test_data.iterrows():
    all_videos += get_videos_between(row['video_start_time'],row['video_end_time'], all_paths)
    
all_videos = sorted(list(set(all_videos)))
paths = [datestring_to_filename(x) for x in all_videos]
del all_paths
del test_data

# Continue here

## Zwischenschritt: Merge all close Tracks of one bee

In [17]:
# RUN THIS IF TRACKS WHERE ALREADY CALCULATED AND SAVED TO AN .csv
# CSV EINLESEN UND SETZEN
tracks = pd.read_csv("all_tracks.csv", sep=";")

In [18]:
def gather_tracks(tracks):
    """
    transform the tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    """

    #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    grouped = tracks.groupby(['bee_id','track_id','video'],  as_index=False)['xpos','ypos','timestamp','zrotation']

    tracks_ml = grouped.aggregate(lambda x: list(x))

    # we don't need track_id anymore
    tracks_ml = tracks_ml.drop('track_id', 1)

    # add a column: convert video name to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video'].apply(lambda x: string_to_timestamp(x))
    
    # we don't need video anymore
    tracks_ml = tracks_ml.drop('video', 1)

    # because aggregated: now multiple timestamps per row: --> rename
    tracks_ml = tracks_ml.rename(columns={'timestamp': 'timestamps'})

    # calculate start time of track by adding timestamp of track (seconds since start of video)
    # to timestamp of video (date)
    tracks_ml['track_start_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[0])
    tracks_ml['track_end_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[-1])
    
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
    
    all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
    all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
    all_paths.sort_values(['video'])
    # get end video
    tracks_ml['video_end_time'] =  tracks_ml['track_end_time'].apply(lambda x: timestamp_to_string(x))
    tracks_ml['video_end_time'] =  tracks_ml[['video_start_time','video_end_time']].apply(lambda x: get_videos_between(x[0],x[1],all_paths)[-1], axis=1)

    return tracks_ml

def merge_tracks(tracks_ml, verbose = False):
    """
    in: 
        tracks_ml: output from gather_tracks(tracks)
    out:
        same df as in, with merged rows
        [bee_id:Float, xpos:[Float], ypos:[Float], zrotation:[Float], timestamps:[Float], video_start_time: String,
        video_end_time: String, track_start_time:Float, track_end_time:Float]
        
    merge tracks of same bee where start and end timestamps are close together
    assume there can not be overlapping tracks
    """
    
    
    # 1. sort: bee_id, start_time
    tracks_ml = tracks_ml.sort_values(['bee_id', 'track_start_time'])

    
    # first convert to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: string_to_timestamp(x))
    
    index = 0
    while(True):
        row = tracks_ml.iloc[index]
        next_row = tracks_ml.iloc[index+1]

        # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
        # merge rows means, next_row is deleted
        if (row['bee_id'] == next_row['bee_id']) and ((next_row['track_start_time'] - row['track_end_time']) < 10):
            
            if verbose:
                print(timestamp_to_string(row['video_start_time']), row['bee_id'], next_row['track_start_time'] - row['track_end_time'])
                print(next_row['video_start_time'] - row['video_start_time'])
                print(timestamp_to_string(row['track_start_time']),timestamp_to_string(row['track_end_time']),timestamp_to_string(row['video_start_time']),row['video_end_time'])
                print(row['timestamps'])
                print(row.name)
                print(timestamp_to_string(next_row['track_start_time']))
                print('----------------------------------')
           
            # update the timestamps of nextrow
            t = next_row['video_start_time'] - row['video_start_time']
            timestamps = list( np.array(next_row['timestamps']) + t)

            # merge xpos, ypos, timestamps lists
            tracks_ml.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
            tracks_ml.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
            tracks_ml.at[row.name,'timestamps'] = row['timestamps']+timestamps

            # update end_time
            tracks_ml.at[row.name,'track_end_time'] = next_row['track_end_time']
            tracks_ml.at[row.name,'video_end_time'] = next_row['video_end_time']

            # delete the merged row (next_row)
            tracks_ml = tracks_ml.drop(next_row.name).copy()

        else:
            index += 1
        
        if index == len(tracks_ml) - 1:
            break
            
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
            
    return tracks_ml

In [19]:
tracks_ml = gather_tracks(tracks)
tracks_ml = merge_tracks(tracks_ml)
tracks_ml = tracks_ml.sort_values(['video_start_time'])

In [20]:
tracks_ml

,bee_id,xpos,ypos,timestamps,zrotation,video_start_time,track_start_time,track_end_time,video_end_time
202,1970.0,"[1175.1580028533936, 1275.0042209625246, 1199....","[100.45368194580078, 252.97906684875488, 264.9...","[1.6, 2.3, 2.8, 3.1, 4.0, 4.1, 4.2, 4.3, 4.6, ...","[-1.5437196000000002, 0.18208171, 1.7509313000...",2018-08-19-02-21-37,1.534638e+09,1.534638e+09,2018-08-19-02-22-40
230,3890.0,"[1275.218282699585, 1286.7158908843996]","[164.32666015625, 214.9285774230957]","[9.2, 9.3]","[-0.34192994, 0.24574603]",2018-08-19-02-21-58,1.534638e+09,1.534638e+09,2018-08-19-02-21-58
197,1697.0,"[449.36813163757324, 449.7386884689331, 449.44...","[162.46553993225098, 162.55523109436038, 149.9...","[3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, ...","[-2.1262455, -2.0971236, -2.1020923, -2.002343...",2018-08-19-02-22-40,1.534638e+09,1.534638e+09,2018-08-19-02-22-40
1,220.0,"[587.6349468231201, 577.2753982543945, 589.496...","[123.45260810852051, 125.9588794708252, 138.67...","[0.4, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6...","[-2.787043, 0.020939333, -0.15247042, 0.053603...",2018-08-19-05-25-51,1.534649e+09,1.534649e+09,2018-08-19-05-25-51
228,3692.0,"[1125.3886528015137, 1075.4727592468264, 1013....","[263.25339698791504, 224.84710693359372, 212.8...","[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0, 1.3]","[2.9821217, 2.9418721000000003, 2.897287, 3.08...",2018-08-19-06-35-41,1.534653e+09,1.534653e+09,2018-08-19-06-35-41
215,2414.0,"[887.616491317749, 762.8647708892822, 650.5874...","[162.65403938293457, 174.80252265930176, 174.8...","[0.0, 0.1, 0.2, 0.3, 0.4, 7.9]","[-2.9935362000000003, -2.6845756, -2.7959702, ...",2018-08-19-07-10-38,1.534655e+09,1.534655e+09,2018-08-19-07-10-38
47,748.0,"[1438.2203845977785, 1375.1870021820068, 1299....","[274.9820747375488, 263.1194038391113, 225.067...","[16.6, 16.7, 16.8, 16.9, 17.0, 17.1, 17.3, 17....","[2.8735619, 2.985191, -2.9177494, 3.0255954, 2...",2018-08-19-07-16-49,1.534656e+09,1.534656e+09,2018-08-19-07-16-49
16,357.0,"[375.14064025878906, 473.86213397979736, 524.1...","[214.47888946533203, 214.77827072143555, 226.9...","[14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14....","[-0.19020759, 0.04539251, 0.12374013, -0.11384...",2018-08-19-07-16-49,1.534656e+09,1.534656e+09,2018-08-19-07-16-49
232,3925.0,"[538.5966873168945, 588.3075313568115, 651.012...","[126.37010765075684, 126.80798149108887, 127.0...","[11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6]","[-0.48655950000000003, -0.23565856, -0.0598887...",2018-08-19-07-47-52,1.534658e+09,1.534658e+09,2018-08-19-07-47-52
35,481.0,"[399.8132972717285, 487.64802932739263, 524.87...","[188.32474899291992, 189.46856498718262, 176.9...","[20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20....","[-0.13244355, -0.2602512, -0.17865844, -0.0478...",2018-08-19-08-21-08,1.534660e+09,1.534660e+09,2018-08-19-08-21-08


In [21]:
# read in the test data csv
test_data = loadGTD("bees_test.csv")

# get all tracks where bee_id, videio_start_time and video_end_time are the same
tracks_ml = pd.merge(tracks_ml,test_data,how='inner',on=['bee_id','video_start_time','video_end_time'])

# we only look at tracks where  in_direction and out_direction are different
tracks_ml = tracks_ml[(tracks_ml['in_direction'] != tracks_ml['out_direction'])]

# we assign movement based on in_ and out_directoin
tracks_ml.loc[(tracks_ml['in_direction'] == 'left') & (tracks_ml['out_direction'] == 'right'),'movement'] = 1
tracks_ml.loc[(tracks_ml['in_direction'] == 'right') & (tracks_ml['out_direction'] == 'left'),'movement'] = 0

tracks_ml.drop('in_direction', 1, inplace=True)
tracks_ml.drop('out_direction', 1, inplace=True)

# Baseline-Algorithmen

## Algorithmus 1: Baseline - Areas as Decider

In [22]:
from math import pi
""" Vorgehen:
Für jedes Bienen-Track Paar
- Prüfe, wo sich die Biene beim ersten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
- Prüfe, wo sich die Biene beim letzten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
Wenn sich Biene in der Mitte befindet, starte neue Routine, die links oder rechts zuordnet
Daraus kann nun abgeleitet werden, wo die Biene reingekommen ist und wo sie rausgegangen ist.
"""
# param: tracks_ml, s. oben
def baseline_alg_classify_bee(tracks_ml):
    def get_direction(xpos, zpos):
        #Helpers
        def is_left(xpos):
            return xpos <= config["left_leaving_area"]*config["px_x_resolution_vid"]
        def is_right(xpos):
            return xpos >= config["px_x_resolution_vid"] - config["right_leaving_area"]*config["px_x_resolution_vid"]
        # Routine, wenn Biene in der Mitte
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        def middle_classifier(zRotation):
            if abs(zRotation) > pi/2:
                return "left"
            else:
                return "right"
            
        # Eintrittsseite festlegen
        if is_left(xpos):
            return "left"
        elif is_right(xpos):
            return "right"
        else:
            return middle_classifier(zpos)
    
    # gets directions return movement as int
    def movement_decider(in_dir, out_dir):
        if in_dir == "left" and out_dir == "right":
            return 1
        elif in_dir == "right" and out_dir == "left":
            return 0
        else:
            return -1
        
    
    pred_movement = np.empty(len(tracks_ml))
    int index = 0
    for row in tracks_ml.itertuples(index=False):
        pred_movement[index] = movement_decider(get_in_direction(row[4][0]), get_out_direction(row[4][-1])
    
    return results
    

SyntaxError: invalid syntax (<ipython-input-22-3277f694bf95>, line 47)

In [ ]:
# Ausgeben
results = baseline_alg_classify_bee(tracks_ml)
results = pd.DataFrame(data=results)
# write ergebnis to csv
results.to_csv('MachineLearnedData.csv', encoding="utf-8", index=False)

## Algorithmus 2: Baseline - zpos as decider

In [ ]:
from math import pi
""" Vorgehen:
Unterschied: Hier wird nicht auf die Position abgestellt, sondern ausschließlich auf die
Richtung, in die die Biene schaut
"""
#results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[], "out_direction":[]}
# param: tracks_ml, s. oben
def baseline_2_alg_classify_bee(tracks_ml):
    def get_in_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "right"
        else:
            return "left"
    def get_out_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "left"
        else:
            return "right"
    
    # "bee_id":[int], "time_in":[String], "os_in":[Float], "time_out":[String],
    # "os_out":[Float], "in_direction":[{"left", "right"}], "out_direction":[{"left", "right"}]
    results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[],
               "out_direction":[]}
        
    for row in tracks_ml.itertuples(index=False):
        results["bee_id"].append(row[0])
        results["time_in"].append(row[5])
        results["os_in"].append(row[-3] - string_to_timestamp(row[5])) #track_start_time - video_start_time
        results["time_out"].append(row[-1])
        results["os_out"].append(row[-2] - string_to_timestamp(row[-1])) #track_end_time - video_end_time
        results["in_direction"].append(get_in_direction(row[4][0]))
        results["out_direction"].append(get_out_direction(row[4][-1]))
    
    return results
    

In [ ]:
# Ausgeben
results = baseline_2_alg_classify_bee(tracks_ml)
results = pd.DataFrame(data=results)
# write ergebnis to csv
results.to_csv('MachineLearnedData.csv', encoding="utf-8", index=False)
results

# Algorithmus 3: Machine Learning

## Vorbereiten Lernvideos, Prüfvideos

### Optional: Tracks um 180° drehen; Tracks horizontal invertieren; Tracks vertikal invertieren (Funktioniert, abhängig von der Wichtigkeit der Zeit als Feature, vielleicht nicht!)

In [ ]:
# Tracks um 180° drehen
track_rotated = tracks_ml


## 3.1 Lernen Logistische Regression

## 3.1 Alg: Logistische Regression

## 3.2 Lernen Decision Tree

## 3.2 Alg Decision Tree

# Vergleichsfunktion Algorithmus mit Labels

In [25]:
import pandas as pd
import os
""" 
Read in two csv files and store in DataFrames. Merge DataFrames to get exact correctness of 
total-, in- and out_direction of bees regarding their time_in and time_out relation  
"""

# delete existing result.csv file to prevent errors
if (os.path.isfile('./result.csv')):
    os.remove("result.csv")

# store data in dataFrame
gtd = loadGTD('bees_test.csv')
mld = pd.read_csv('MachineLearnedData.csv')

# change column name to merge dataFrames
gtd.rename(columns = {'video_start_time':'time_in', 'video_end_time':'time_out'}, inplace=True)

# delete dublications
mld = mld.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')
gtd = gtd.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')

# change DT from float to int to have the same DT like gtd
mld.bee_id = mld.bee_id.astype('int32')

# merge to get 100% correct matches
mergeTotalCorrect = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out', 'in_direction', 'out_direction'], how='inner')
#merge to get in_direction correct
mergeInDirection = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'in_direction'], how='inner')
#merge to get out_direction correct
mergeOutDirection = pd.merge(gtd, mld, on=['bee_id', 'time_out', 'out_direction'], how='inner')
# merge to get comparable bees to get all found bees
mergeableBees = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out'], how='inner')

# iterate mergeableBees for visualization
result = pd.DataFrame()
result['bee_id'] = mergeableBees['bee_id'] 
result['in_equals'] = mergeableBees['in_direction_x'] == mergeableBees['in_direction_y']
result['out_equals'] = mergeableBees['out_direction_x'] == mergeableBees['out_direction_y']
#result['movement'] = result['in_equals'].bool() == True and result['out_equals'].bool() == False

# store to result.csv
result.to_csv('result.csv', encoding='utf-8', index=False)

# calculate statitics
inCorrect = round((result['in_equals'].sum()/len(result) * 100), 2)
outCorrect = round((result['out_equals'].sum()/len(result) * 100), 2)
allCorrect = round(((result['in_equals'] == result['out_equals']).sum()/len(result) * 100), 2)
aveCorrect = round(((allCorrect + inCorrect + outCorrect) / 3), 2)

# print statistics
try:
    print(len(gtd), 'entries in GroundTruthData.', len(mergeableBees), 'entries compareable in MachineLearnedData: ', round((len(mergeableBees)/len(gtd))*100, 2), '%' )
    print(allCorrect, "% total correctness.")
    print(inCorrect, "% Ingoing correctness.")
    print(outCorrect, "% Outgoing correctness.")
    print(aveCorrect, "% Average correctness.")
except ZeroDivisionError:
    print("Nothing to compare!")

# end compare algorithm
print('\nCheck result.csv file or hit \'result\' for more details.')
print('Use DataFrames for deeper information: \n\'mergeTotalCorrect\',\'mergeInCorrect\',\'mergeOutCorrect\',\'mergeableBees\' ')


103 entries in GroundTruthData. 44 entries compareable in MachineLearnedData:  42.72 %
70.45 % total correctness.
79.55 % Ingoing correctness.
90.91 % Outgoing correctness.
80.3 % Average correctness.

Check result.csv file or hit 'result' for more details.
Use DataFrames for deeper information: 
'mergeTotalCorrect','mergeInCorrect','mergeOutCorrect','mergeableBees' 


In [26]:
tracks_ml

,bee_id,xpos,ypos,timestamps,zrotation,video_start_time,track_start_time,track_end_time,video_end_time,movement
1,220.0,"[587.6349468231201, 577.2753982543945, 589.496...","[123.45260810852051, 125.9588794708252, 138.67...","[0.4, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6...","[-2.787043, 0.020939333, -0.15247042, 0.053603...",2018-08-19-05-25-51,1.534649e+09,1.534649e+09,2018-08-19-05-25-51,1.0
2,3925.0,"[538.5966873168945, 588.3075313568115, 651.012...","[126.37010765075684, 126.80798149108887, 127.0...","[11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6]","[-0.48655950000000003, -0.23565856, -0.0598887...",2018-08-19-07-47-52,1.534658e+09,1.534658e+09,2018-08-19-07-47-52,1.0
3,388.0,"[1375.2675552368164, 1313.4858207702634, 1225....","[239.0948257446289, 238.7294406890869, 237.849...","[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]","[2.9390156000000003, 2.9522842999999996, -2.97...",2018-08-19-08-29-06,1.534660e+09,1.534660e+09,2018-08-19-08-29-06,0.0
4,1066.0,"[1413.0752410888672, 1350.3371124267578, 1325....","[251.15822410583496, 238.2443084716797, 188.21...","[1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, ...","[2.8809676, 3.0900168, -2.7678201000000002, -3...",2018-08-19-09-28-45,1.534664e+09,1.534664e+09,2018-08-19-09-28-45,0.0
5,446.0,"[477.1684074401856, 526.4452629089354, 601.512...","[151.28521347045896, 152.16980743408203, 138.8...","[20.6, 20.7, 20.9, 21.0, 21.1, 21.2, 21.3, 21....","[0.029027572, 0.03262988, 0.072178215]",2018-08-19-09-47-39,1.534665e+09,1.534665e+09,2018-08-19-09-48-00,1.0
6,1568.0,"[325.2653713226318, 375.9219398498535, 413.437...","[238.9930763244629, 213.9500617980957, 214.077...","[13.8, 14.0, 14.2, 14.9, 15.0, 15.1, 15.2, 15....","[0.39186668, 0.10911903, 0.016913554, 0.301830...",2018-08-19-09-47-39,1.534665e+09,1.534665e+09,2018-08-19-09-48-00,1.0
7,420.0,"[1075.0250930786133, 637.5782222747803, 538.01...","[187.6749038696289, 211.6030120849609, 210.762...","[13.6, 14.0, 14.1, 14.2]","[2.9731865, 2.9255147, 2.7518196, 2.5865266]",2018-08-19-09-48-00,1.534665e+09,1.534665e+09,2018-08-19-09-48-00,0.0
9,1086.0,"[625.8092365264893, 712.9416446685791, 786.658...","[175.7629508972168, 188.81119537353518, 201.19...","[11.7, 11.8, 11.9, 12.0, 12.1, 12.4]","[-0.14008899, -0.03211413, 0.06833663, -0.2839...",2018-08-19-09-55-49,1.534665e+09,1.534665e+09,2018-08-19-09-55-49,1.0
12,830.0,"[325.3576316833496, 349.8817691802978, 474.947...","[226.56576347351074, 201.92008781433103, 176.8...","[7.7, 7.8, 8.0, 8.1, 8.3, 8.4, 8.5, 8.6, 8.7, ...","[-0.056764938, -0.11777138, -0.30179748, -0.17...",2018-08-19-12-04-59,1.534673e+09,1.534673e+09,2018-08-19-12-04-59,1.0
13,3894.0,"[861.9392747879027, 561.8118915557861, 512.620...","[136.67405700683594, 161.2934045791626, 125.23...","[1.3, 1.6, 1.7, 1.8]","[-3.001254, -2.4899282000000005, -2.3435442, -...",2018-08-19-12-04-59,1.534673e+09,1.534673e+09,2018-08-19-12-04-59,0.0
